# Multi Agent Misconceptions

### Todo's

In [1]:
# TODO: Making the code run properly(debuging the FAISS code??)
# TODO: Figuring out how to handle the inconsistency between training data and prediction results.
# TODO: Trying more advanced structure.
# TODO: Use streamlit fo launching the app.

### Imports

In [2]:
import os
import dspy

from dotenv import load_dotenv
from dspy.evaluate import Evaluate
from colorama import Fore, Style, init
from dspy.teleprompt import BootstrapFewShot

from src.agents import Agent
from src.evaluation import evaluate_answers
from src.predict_model import ExchangeOfThought
from src.dataloader import load_data, load_misconceptions

In [3]:
# Initialize colorama
init(autoreset=True)

# Load environment variables
load_dotenv()

# Configure OpenAI API
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
if not OPENAI_API_KEY:
    raise EnvironmentError(
        "OPENAI_API_KEY not found in environment variables.")
os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY
lm = dspy.LM('openai/gpt-4o-mini')
dspy.configure(lm=lm)

### Load Data

In [4]:
# Misconception mapping
misconceptions = load_misconceptions('data/misconception_mapping.csv')

# Load training and test sets
train_data = load_data('data/train.csv')
test_data = load_data('data/train.csv', is_test=True)

In [ ]:
# Set up Agents
agent_a = Agent(name="Agent A")
agent_b = Agent(name="Agent B")
agent_c = Agent(name="Agent C")

# evaluate
evaluate_program = Evaluate(devset=test_data, metric=evaluate_answers,
                            num_threads=8, display_progress=True, display_table=10)
predict = ExchangeOfThought(
    agent_a, agent_b, agent_c, rounds=3, mode="Report")
eval1 = evaluate_program(predict)

predict.save('./result/compiled_model.pkl')

# compile
# teleprompter = BootstrapFewShot(
#     metric=evaluate_answers, max_labeled_demos=1)
# compiled_predictor = teleprompter.compile(predict, trainset=train_data)

# evaluate again
# eval_compiled = evaluate_program(compiled_predictor)
# compiled_predictor.save('./compiled_model.pkl')
# print(eval_compiled)

Average Metric: 4.00 / 19 (21.1%): 100%|██████████| 19/19 [03:46<00:00, 11.92s/it]


2024/11/26 16:26:48 INFO dspy.evaluate.evaluate: Average Metric: 4 / 19 (21.1%)


,example_question,example_answer,reasoning,qustion,pred_answer,pred_question,evaluate_answers
0,The question is: This displacement-time graph shows \( 10 \) secon...,"[747.0, 1731.0, 1585.0, nan]",The question requires calculating the distance based on the speed ...,The question is: This displacement-time graph shows \( 10 \) secon...,{ 'A': 'MisconceptionA: This option may arise from a misunderstand...,The question is: This displacement-time graph shows \( 10 \) secon...,
1,The question is: \( 0.5=\frac{1}{2}=? \% \) And here is the possib...,"[1786.0, nan, 292.0, nan]",The question asks for the percentage equivalent of the fraction \(...,The question is: \( 0.5=\frac{1}{2}=? \% \),"{ 'A': 'Miscalculation or confusion with a different fraction', 'B...",The question is: \( 0.5=\frac{1}{2}=? \% \) And here is the possib...,
2,"The question is: Simplify, if possible: \( a \times 4 \) And here ...","[nan, 267.0, 400.0, nan]",The question asks to simplify the expression \( a \times 4 \). The...,"The question is: Simplify, if possible: \( a \times 4 \) And here ...","{ 'A': NaN, 'B': 'Misconception: Suggests a misunderstanding of th...","The question is: Simplify, if possible: \( a \times 4 \) And here ...",
3,"The question is: The formula for the area of a triangle, \( A \), ...","[1114.0, 133.0, nan, nan]",The question requires rearranging the formula for the area of a tr...,"The question is: The formula for the area of a triangle, \( A \), ...","'A': 'MisconceptionA',\n'B': 'MisconceptionB',\n'C': NaN,\n'D': 'M...","The question is: The formula for the area of a triangle, \( A \), ...",
4,The question is: ![A speed-time graph. The vertical axis is labell...,"[790.0, nan, nan, nan]",To find the speed \( v \) given that the total distance travelled ...,The question is: ![A speed-time graph. The vertical axis is labell...,'A': 'This may arise from incorrectly calculating the average spee...,The question is: ![A speed-time graph. The vertical axis is labell...,
5,The question is: Complete this statement: \( 4 \) litres \( = \)__...,"[273.0, nan, 474.0, 1863.0]","The question requires converting litres to cubic centimeters, wher...",The question is: Complete this statement:\n\( 4 \) litres \( = \)_...,'A': 'MisconceptionA: This option suggests that \( 4 \) litres is ...,The question is: Complete this statement: \( 4 \) litres \( = \)__...,✔️ [True]
6,The question is: The formula for finding the average speed is \( \...,"[nan, nan, 1540.0, nan]",The question is asking which of the provided options are not corre...,The question is: The formula for finding the average speed is \( \...,"'A': NaN, 'B': 'Misconception: This option incorrectly states the ...",The question is: The formula for finding the average speed is \( \...,
7,The question is: What is the median of the following numbers? \( 2...,"[732.0, nan, 2198.0, 461.0]",The median is the middle value of a sorted list of numbers. In thi...,The question is: What is the median of the following numbers? \( 2...,"'A': 'MisconceptionA',\n'B': NaN,\n'C': 'MisconceptionC',\n'D': 'M...",The question is: What is the median of the following numbers? \( 2...,
8,The question is: Which of the following situations would a line gr...,"[1029.0, 1029.0, nan, 1029.0]",A line graph is most appropriate for displaying data that changes ...,The question is: Which of the following situations would a line gr...,'A': 'MisconceptionA: A line graph is suitable for showing relatio...,The question is: Which of the following situations would a line gr...,✔️ [True]
9,The question is: This graph shows the displacement of a ball from ...,"[378.0, 378.0, nan, 1887.0]",The question requires interpreting a displacement versus time grap...,The question is: This graph shows the displacement of a ball from ...,'A': 'Misconception: The option \( 0.5 \ \text{m} \) may arise fro...,The question is: This graph shows the displacement of a ball from ...,
